In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb
!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [

In [ ]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, InvalidArgumentException
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, TimeoutException, InvalidArgumentException


chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Disable images as there are alot
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

# WebDriver
driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(180)
wait = WebDriverWait(driver, 15)

# Handle cookie popup
def handle_popups():
    try:
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'coi-banner__accept')))
        accept_cookies_button.click()
        print("Accepted cookies.")
    except Exception as e:
        print(f"Could not find or accept cookies: {e}")

def scroll_and_trigger_login():
    try:
        if driver.find_elements(By.XPATH, "//body[contains(@class, 'popup-open')]"):
            print("Login popup is already visible. Proceeding with iframe switch and login.")
        else:
            print("Scrolling to the bottom to trigger the login...")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)

            # find and click login btn
            login_trigger = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'mu-dialog__show')))
            login_trigger.click()
            print("Clicked 'Log ind' to trigger the login dialog.")

            # wait until the login popup is fully open (can detect in class)
            wait.until(EC.presence_of_element_located((By.XPATH, "//body[contains(@class, 'popup-open')]")))
            print("Login popup is now visible.")

        #
        try:
            iframe = driver.find_element(By.ID, "paywall-iframe")
            driver.switch_to.frame(iframe)
            print("Switched to iframe with ID: paywall-iframe")
        except NoSuchElementException:
            print("No iframe found, unable to proceed with login.")
            return
        try:
            email_input = wait.until(EC.element_to_be_clickable((By.ID, 'email')))
            password_input = wait.until(EC.element_to_be_clickable((By.ID, 'field_password')))
            print("Located email and password fields.")
        except Exception as e:
            print(f"Error locating login fields: {e}")
            return

        time.sleep(2)

        driver.execute_script("arguments[0].value='norg@itu.dk';", email_input)
        driver.execute_script("arguments[0].value='xxxx';", password_input)
        print("Filled in email and password.")

        submit_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type=\"submit\"]')))
        driver.execute_script("arguments[0].scrollIntoView(true);", submit_button)

        driver.execute_script("arguments[0].click();", submit_button)
        print("Submitted login form.")

        wait.until(EC.url_contains('/artikel'))
        print("Redirected back to the article after login.")

        handle_popups()

        driver.switch_to.default_content()

    except Exception as e:
        print(f"Login failed: {e}")



# url is sometimes ing, other times version2 an third times radar
def get_full_url(article_url):
    if article_url.startswith('/'):
        return f"https://ing.dk{article_url}"
    return article_url

# check for the paywall
def handle_paywall(full_url):
    try:
        paywall_form = driver.find_element(By.CLASS_NAME, 'plus-paywall-form')
        print(f"Paywall detected for {full_url}. Attempting to trigger login...")

        try:
            login_trigger = driver.find_element(By.CSS_SELECTOR, "span.mu-dialog__show.mu-dialog--login")
            driver.execute_script("arguments[0].click();", login_trigger)
            print("Login button clicked, waiting for popup...")

            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'mu-dialog')))
            print("Login popup is now visible.")

            scroll_and_trigger_login()

            return True

        except Exception as e:
            print(f"Error triggering login: {e}")
            return False

    except NoSuchElementException:
        print("No paywall detected.")
        return True

# scrape individual article
def scrape_article(article_url):
    full_url = get_full_url(article_url)
    print(f"Scraping article: {full_url}")

    try:
        driver.get(full_url)
        handle_popups()
    except InvalidArgumentException as e:
        print(f"Error with URL {full_url}: {e}")
        return None

    time.sleep(2)

    if not handle_paywall(full_url):
        return None

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    try:
        title = soup.find('h1', class_='article-header--title').get_text()
    except:
        title = 'No title'

    try:
        subheader = soup.find('div', class_='article-summary').get_text()
    except:
        subheader = 'No subheader'

    try:
        date = soup.find('span', class_='additional-info--publication').get_text()
    except:
        date = 'No date'

    try:
        content_div = soup.find('div', class_='block-entity-fieldnodebody')
        paragraphs = content_div.find_all('p')
        content = " ".join([p.get_text() for p in paragraphs])
    except:
        content = 'No content'

    return {
        'url': full_url,
        'title': title,
        'subheader': subheader,
        'date': date,
        'content': content
    }


def scrape_articles(search_term, max_articles=10):
    article_count = 0
    page = 0
    all_articles = []

    while article_count < max_articles:
        search_url = f"https://ing.dk/sog?query={search_term}&page={page}"

        driver.get(search_url)
        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.find_all('article', class_='list-item')

        if not articles:
            print("No more articles found.")
            break

        for article in articles:
            if article_count >= max_articles:
                break

            article_url = article.find('a')['href']

            article_data = scrape_article(article_url)
            if article_data:
                all_articles.append(article_data)
                article_count += 1
                print(f"Scraped {article_count} articles.")

        page += 1

    return all_articles

search_term = ['ansigtsgenkendelse', 'facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']

scraped_articles = scrape_articles(search_term, max_articles=300)


df = pd.DataFrame(scraped_articles)
df.to_csv('ing_scraped_articles.csv', index=False)
print(df.head())


driver.quit()

Scraping article: https://radar.dk/artikel/er-vi-midt-i-et-tech-kup-det-mener-fremtraedende-stanford-direktoer-som-er-klar-til-forbyde-nye
Accepted cookies.
Paywall detected for https://radar.dk/artikel/er-vi-midt-i-et-tech-kup-det-mener-fremtraedende-stanford-direktoer-som-er-klar-til-forbyde-nye. Attempting to trigger login...
Login button clicked, waiting for popup...
Login popup is now visible.
Login popup is already visible. Proceeding with iframe switch and login.
Switched to iframe with ID: paywall-iframe
Located email and password fields.
Filled in email and password.
Submitted login form.
Redirected back to the article after login.
Could not find or accept cookies: Message: 
Stacktrace:
#0 0x5b7e9bdc902a <unknown>
#1 0x5b7e9baaf5e0 <unknown>
#2 0x5b7e9bafebe8 <unknown>
#3 0x5b7e9bafee81 <unknown>
#4 0x5b7e9bb458c4 <unknown>
#5 0x5b7e9bb23b4d <unknown>
#6 0x5b7e9bb42d7d <unknown>
#7 0x5b7e9bb238c3 <unknown>
#8 0x5b7e9baf16b3 <unknown>
#9 0x5b7e9baf268e <unknown>
#10 0x5b7e9bd93

Did consistenly have issues with this one, as Ingengiøren both consists of Ing, radar and version 2, switching back and forth on other websites. ultimately, it resulted in not all article content being scraped and a few did thus not have the content here. In the end, i wound up inputting for these manually as it wa a manageable amount.

In [ ]:
!pip install openpyxl


In [ ]:
df.to_excel('Ing_articles.xlsx', index=False)


In [ ]:
from google.colab import files
files.download('Ing_articles.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive')
df = pd.read_excel('/content/gdrive/MyDrive/Thesis/Data/Ing_articles.xlsx')
df

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,url,title,subheader,date,content
0,https://radar.dk/artikel/er-vi-midt-i-et-tech-...,Er vi midt i et tech-kup? Det mener fremtræden...,Big Tech er ved at antage form af statens arvt...,25. september kl. 07:54,No content
1,https://www.version2.dk/artikel/politibetjente...,Politibetjente frygter uklar lov om brug af an...,Det kan potentielt få ansættelsesretlige konse...,23. september kl. 05:30,No content
2,https://ing.dk/artikel/leder-af-ai-center-tekn...,Leder af AI-center: Teknologi til ansigtsgenke...,Uden juridisk kontrol og repræsentative datasæ...,11. september kl. 15:29,No content
3,https://radar.dk/artikel/forskere-ansigtsgenke...,Forskere: Ansigtsgenkendelse er langtfra præci...,I test er nogle af verdens bedste ansigtsgenke...,10. september kl. 05:00,No content
4,https://www.version2.dk/artikel/fck-staar-paa-...,FCK står på spring med ansigtsgenkendelse,Fodboldklubben har ventet i fem måneder på at ...,10. september kl. 05:35,No content
...,...,...,...,...,...
174,https://ing.dk/artikel/eksperter-oeget-overvaa...,Eksperter: Øget overvågning ved biometriske pas,Identitetstyveri og øget overvågning kan være ...,27. januar 2005 kl. 00:00,"Danmark og resten af EU skal have nye pas, hvo..."
175,https://ing.dk/artikel/kun-fantasien-saetter-g...,Kun fantasien sætter grænsen,No subheader,16. april 2004 kl. 00:00,En minimaliseret videoudlejningsbutik er opsti...
176,https://ing.dk/artikel/irisskanning-rykker-ind...,Irisskanning rykker ind på skrivebordet,Fortidens kostbare biometriske systemer til ad...,7. juni 2002 kl. 00:00,"Øjets regnbuehinde, den såkaldte iris, er endn..."
177,https://ing.dk/artikel/systemudviklere-skal-va...,Systemudviklere skal være sociologer,"Traditionel IT-systemudvikling er død, mener d...",19. maj 2002 kl. 12:00,Phil Agre er Internetveteran. Han har været ak...


In [ ]:
df['search_term'] = 'ansigtsgenkendelse'


In [ ]:
# fixing dates
month_mapping = {
    'januar': 'January', 'februar': 'February', 'marts': 'March', 'april': 'April',
    'maj': 'May', 'juni': 'June', 'juli': 'July', 'august': 'August',
    'september': 'September', 'oktober': 'October', 'november': 'November', 'december': 'December'
}

def clean_date(date_str):
    if 'kl.' in date_str:
        cleaned_date = date_str.split('kl.')[0].strip()
    else:
        cleaned_date = date_str

    for danish_month, english_month in month_mapping.items():
        cleaned_date = cleaned_date.replace(danish_month, english_month)

    if not any(char.isdigit() for char in cleaned_date.split()[-1]):
        cleaned_date += ' 2024'

    return cleaned_date

df['cleaned_date'] = df['date'].apply(clean_date)

df['date'] = pd.to_datetime(df['cleaned_date'], format='%d. %B %Y', errors='coerce')
df

,url,title,subheader,date,content,search_term,cleaned_date
0,https://radar.dk/artikel/er-vi-midt-i-et-tech-...,Er vi midt i et tech-kup? Det mener fremtræden...,Big Tech er ved at antage form af statens arvt...,2024-09-25,No content,ansigtsgenkendelse,25. September 2024
1,https://www.version2.dk/artikel/politibetjente...,Politibetjente frygter uklar lov om brug af an...,Det kan potentielt få ansættelsesretlige konse...,2024-09-23,No content,ansigtsgenkendelse,23. September 2024
2,https://ing.dk/artikel/leder-af-ai-center-tekn...,Leder af AI-center: Teknologi til ansigtsgenke...,Uden juridisk kontrol og repræsentative datasæ...,2024-09-11,No content,ansigtsgenkendelse,11. September 2024
3,https://radar.dk/artikel/forskere-ansigtsgenke...,Forskere: Ansigtsgenkendelse er langtfra præci...,I test er nogle af verdens bedste ansigtsgenke...,2024-09-10,No content,ansigtsgenkendelse,10. September 2024
4,https://www.version2.dk/artikel/fck-staar-paa-...,FCK står på spring med ansigtsgenkendelse,Fodboldklubben har ventet i fem måneder på at ...,2024-09-10,No content,ansigtsgenkendelse,10. September 2024
...,...,...,...,...,...,...,...
174,https://ing.dk/artikel/eksperter-oeget-overvaa...,Eksperter: Øget overvågning ved biometriske pas,Identitetstyveri og øget overvågning kan være ...,2005-01-27,"Danmark og resten af EU skal have nye pas, hvo...",ansigtsgenkendelse,27. January 2005
175,https://ing.dk/artikel/kun-fantasien-saetter-g...,Kun fantasien sætter grænsen,No subheader,2004-04-16,En minimaliseret videoudlejningsbutik er opsti...,ansigtsgenkendelse,16. April 2004
176,https://ing.dk/artikel/irisskanning-rykker-ind...,Irisskanning rykker ind på skrivebordet,Fortidens kostbare biometriske systemer til ad...,2002-06-07,"Øjets regnbuehinde, den såkaldte iris, er endn...",ansigtsgenkendelse,7. June 2002
177,https://ing.dk/artikel/systemudviklere-skal-va...,Systemudviklere skal være sociologer,"Traditionel IT-systemudvikling er død, mener d...",2002-05-19,Phil Agre er Internetveteran. Han har været ak...,ansigtsgenkendelse,19. May 2002


In [ ]:
df['merged_content'] = df['title'] + ' ' + df['subheader'] + ' ' + df['content']
df

,url,title,subheader,date,content,search_term,cleaned_date,merged_content
0,https://radar.dk/artikel/er-vi-midt-i-et-tech-...,Er vi midt i et tech-kup? Det mener fremtræden...,Big Tech er ved at antage form af statens arvt...,2024-09-25,No content,ansigtsgenkendelse,25. September 2024,Er vi midt i et tech-kup? Det mener fremtræden...
1,https://www.version2.dk/artikel/politibetjente...,Politibetjente frygter uklar lov om brug af an...,Det kan potentielt få ansættelsesretlige konse...,2024-09-23,No content,ansigtsgenkendelse,23. September 2024,Politibetjente frygter uklar lov om brug af an...
2,https://ing.dk/artikel/leder-af-ai-center-tekn...,Leder af AI-center: Teknologi til ansigtsgenke...,Uden juridisk kontrol og repræsentative datasæ...,2024-09-11,No content,ansigtsgenkendelse,11. September 2024,Leder af AI-center: Teknologi til ansigtsgenke...
3,https://radar.dk/artikel/forskere-ansigtsgenke...,Forskere: Ansigtsgenkendelse er langtfra præci...,I test er nogle af verdens bedste ansigtsgenke...,2024-09-10,No content,ansigtsgenkendelse,10. September 2024,Forskere: Ansigtsgenkendelse er langtfra præci...
4,https://www.version2.dk/artikel/fck-staar-paa-...,FCK står på spring med ansigtsgenkendelse,Fodboldklubben har ventet i fem måneder på at ...,2024-09-10,No content,ansigtsgenkendelse,10. September 2024,FCK står på spring med ansigtsgenkendelse Fodb...
...,...,...,...,...,...,...,...,...
174,https://ing.dk/artikel/eksperter-oeget-overvaa...,Eksperter: Øget overvågning ved biometriske pas,Identitetstyveri og øget overvågning kan være ...,2005-01-27,"Danmark og resten af EU skal have nye pas, hvo...",ansigtsgenkendelse,27. January 2005,Eksperter: Øget overvågning ved biometriske pa...
175,https://ing.dk/artikel/kun-fantasien-saetter-g...,Kun fantasien sætter grænsen,No subheader,2004-04-16,En minimaliseret videoudlejningsbutik er opsti...,ansigtsgenkendelse,16. April 2004,Kun fantasien sætter grænsen No subheader En m...
176,https://ing.dk/artikel/irisskanning-rykker-ind...,Irisskanning rykker ind på skrivebordet,Fortidens kostbare biometriske systemer til ad...,2002-06-07,"Øjets regnbuehinde, den såkaldte iris, er endn...",ansigtsgenkendelse,7. June 2002,Irisskanning rykker ind på skrivebordet Fortid...
177,https://ing.dk/artikel/systemudviklere-skal-va...,Systemudviklere skal være sociologer,"Traditionel IT-systemudvikling er død, mener d...",2002-05-19,Phil Agre er Internetveteran. Han har været ak...,ansigtsgenkendelse,19. May 2002,Systemudviklere skal være sociologer Tradition...


In [ ]:
from google.colab import files

df.to_csv('ing_articles_with_dates.csv', index=False)
files.download('ing_articles_with_dates.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>